In [12]:
# -*- coding:utf-8 -*-
import openpyxl
from bs4 import BeautifulSoup
from urllib.request import urlopen
import csv, re, requests
import time

# 엑셀파일 열기
filename = "test.xlsx"
book = openpyxl.load_workbook(filename)

# 맨 앞의 시트 추출하기
sheet = book.worksheets[0]

# 시트의 각 행을 순서대로 추출하기
data = []
for row in sheet.rows:
    data.append([row[0].value, row[1].value,row[2].value])

del data[0]

for i in range(len(data)):
    print(data[i])


['NT930QCG-K58A', '박문규', '20.11.03']
['NT350XCR-AD5WA', '임선정', '20.11.05']
['15ZD90N-VX50K', '권규현', '20.10.29']
['15UD40N-GX36K', '이승호', '20.10.29']
['15UD40N-GX36K', '박문규', '20.10.28']
['75ATD430N-66h', '김민창', '20.10.29']
['NT350XCR-AD5WA', '박문규', '20.10.29']
['NT950XCR-G58A', '박문규', '20.10.29']
['17ZD90N-VX70K', '권규현', '20.10.29']
['NT950QCT-A58A', '권규현', '20.10.29']
['15ZD90N-HX56K', '임선정', '20.10.29']
['17ZD90N-VX50K', '박문규', '20.10.29']
['NT950XCR-A58A', '임선정', '20.10.29']
['NT950XCJ-X716A', '권규현', '20.10.29']
[None, None, None]


# Beautiful 빠르나 결국 URL당 접근하면 느려지기때문에 selenium 검색과 시간 비교

In [73]:
start = time.time()  # 시작 시간 저장
result=[]
del data[5]
print(data)

for i in range(len(data)):
    if data[i][0]== None:
        break
    else:
        url = 'https://search.shopping.naver.com/search/all?query=' + data[i][0]
        print(url)
        req = requests.get(url)   # 크롤링할 페이지 url로 요청 보내기 , req를 출력하면 statuscode만 출력됨
        html = req.text                # html 코드를 가져온다!
        soup = BeautifulSoup(html, 'html.parser')
        if soup.find('span', class_="price_num__2WUXn").text == None:
                         result.append("해당상품없음")
        else:
            result.append(soup.find('span', class_="price_num__2WUXn").text)
# 작업 코드
 
 
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
print(result)

[['NT930QCG-K58A', '박문규', '20.11.03'], ['NT350XCR-AD5WA', '임선정', '20.11.05'], ['15ZD90N-VX50K', '권규현', '20.10.29'], ['15UD40N-GX36K', '이승호', '20.10.29'], ['15UD40N-GX36K', '박문규', '20.10.28'], ['17ZD90N-VX70K', '권규현', '20.10.29'], ['NT950QCT-A58A', '권규현', '20.10.29'], ['15ZD90N-HX56K', '임선정', '20.10.29'], ['17ZD90N-VX50K', '박문규', '20.10.29'], ['NT950XCR-A58A', '임선정', '20.10.29'], ['NT950XCJ-X716A', '권규현', '20.10.29'], [None, None, None]]
https://search.shopping.naver.com/search/all?query=NT930QCG-K58A
https://search.shopping.naver.com/search/all?query=NT350XCR-AD5WA
https://search.shopping.naver.com/search/all?query=15ZD90N-VX50K
https://search.shopping.naver.com/search/all?query=15UD40N-GX36K
https://search.shopping.naver.com/search/all?query=15UD40N-GX36K
https://search.shopping.naver.com/search/all?query=17ZD90N-VX70K
https://search.shopping.naver.com/search/all?query=NT950QCT-A58A
https://search.shopping.naver.com/search/all?query=15ZD90N-HX56K
https://search.shopping.naver.com/sear

In [47]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
start = time.time()  # 시작 시간 저장

#Chrome 웹드라이버 생성
driver=webdriver.Chrome()

driver.get("https://shopping.naver.com/")
assert "네이버쇼핑" in driver.title

for i in range(len(data)):
    if data[i][0]== None:
        break
    else:
        elem=driver.find_element_by_xpath('//*[@id="autocompleteWrapper"]/input[1]')
        elem.send_keys(data[i][0])
        elem.submit()
        driver.find_element_by_xpath('//*[@id="autocompleteWrapper"]/a[2]').click()
        driver.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div/div[2]/div/h1/a').click()
      
        
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 22.828194618225098


### beautiful  = 모든 url을 열고 get  // 셀리니움  = 하나 url을 열고 input에 원하는 데이터만 바꿔줌 이다.
### 처음생각했을때는 모든 url을 열고 get하는 방식이 느릴것이라고 예상했지만, 셀레니움이 확연하게 느렸다. 확실히 chrome을 키고 네트워크가 로딩하는 과정에서 bandwidth 로인한 cost가 있는거 같다.